<a href="https://colab.research.google.com/github/Tiru-Kaggundi/Trade_Stuff/blob/main/ITS_specialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import pandas as pd

In [100]:
names = pd.read_excel("/content/ITS_Specialization_region_products.xlsx", sheet_name='Territories')
names_list = names[["Serial_no", "Name_of_officer"]]
names_list.Serial_no.astype(str)
names_dict = dict(names_list.values)

In [101]:
territories = pd.read_excel("/content/ITS_Specialization_region_products.xlsx", sheet_name='Territories', header=0, index_col=2)
territories.drop(territories.columns[[0, 1, 2]], axis=1, inplace=True)
territories.fillna(0, inplace=True)
territories.drop(['Any other (please specify)', 'Remarks (if any)'], axis=1, inplace=True)

In [102]:
products = pd.read_excel("/content/ITS_Specialization_region_products.xlsx", sheet_name='Product group', header=0, index_col=2)
products.drop(products.columns[[0, 1, 2]], axis=1, inplace=True)
products.fillna(0, inplace=True)
products.drop(['Any other (please specify)', 'Remarks (if any)'], axis=1, inplace=True)

In [103]:
list_of_officers_region_index = territories.index[territories['NAFTA'] != 0].tolist()
list_of_officers_products_index = products.index[products['Agri - fruits and vegetables, cereals, foodgrain, oilseeds'] != 0].tolist()
k = [names_dict[item] for item in list_of_officers_region_index]


In [105]:
regions_list = {}
sectors_list = {}

for region in territories.columns:
  list_of_officers_region_index = territories.index[territories[region] != 0].tolist()
  list_of_officers = [names_dict[item] for item in list_of_officers_region_index]
  regions_list[region] = list_of_officers

for sector in products.columns:
  list_of_officers_products_index = products.index[products[sector] != 0].tolist()
  list_of_officers = [names_dict[item] for item in list_of_officers_products_index]
  sectors_list[sector] = list_of_officers


In [207]:
summary = pd.read_excel("/content/Summary_ITS_specialisation.xlsx", sheet_name="Summary", header=0, index_col=0)
summary.fillna('', inplace=True)
final = pd.concat([products, territories], axis=1)
index_values = summary.index.to_list()
index_values.append("List of officers")
index_values

['Agri - fruits and vegetables, cereals, foodgrain, oilseeds',
 'Agri - dairy and processed foods + Agri - speciality, botanical, lac and gum, medicinal and herbal ',
 'Agri - Livestock, marine',
 'Basic Chemicals and Dyes- organic and inorganic compounds including Petroleum products',
 'Pharma - Bulk drugs/API, chemical formulations, research and development, medical and surgical instruments',
 'Plastic, Rubber and their products',
 'Leather and leather products',
 'Handicraft items, Handloom including carpets (both hand and machine made), Sports goods',
 'Textile and apparel - natural fibre based (cotton, silk, wool)',
 'Textile and Apparel  - synthetic fibre based - including technical and sportswear',
 'Gems and Jewelry',
 'Automobiles and auto anicilliaries - including E-vehicles and parts',
 'Engineering goods - non automotive (aeroplanes/aircraft, heavy machinery, machine tools, boilers etc)',
 'Electronics - communication (cellphones, mobile phones, internet, routers, including

In [208]:
sum_list = {}
for item in range(final.shape[0]): 
   sum_list[str(item + 1.0)] = dict(final.iloc[item])

In [209]:
rows = list(summary.index.values)
columns = list(summary.columns)

In [210]:
def process_officer(x, names_dict, dict_x, rows, columns):
  '''
  function that processes the officers and places the tick
  '''
  # delete existing values to prevent adding to existing values
  summary.fillna('', inplace=True)
  for sector in rows:
    if dict_x[sector]:
      for region in columns:
        if dict_x[region]:
          if summary.loc[sector][region] is '':
            summary.loc[sector][region] = names_dict[float(x)]
          else:
            summary.loc[sector][region] += ", " + names_dict[float(x)]

In [211]:
for item in sum_list:
  process_officer(item, names_dict, sum_list[item], rows, columns)

In [212]:
summary["List of officers interested"] = pd.Series(sectors_list).values
#summary.loc["List of officers interested"] = pd.Series(regions_list).values
#new = pd.Series(regions_list)
summary = summary.append(pd.Series(regions_list), ignore_index=True)
summary.index = index_values

In [214]:
summary.to_excel("/content/Summary_final.xlsx", header=True, index=True)